In [1]:
import mlflow
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

#### Fix depth, vary $\lambda$

In [2]:
layers = 3
depth = 4
epochs = 50
region = 'southeast_us'
lams = [0.1, 0.01] #[1.0, 0.1, 0.01]
# run experiment
for lam in tqdm(lams):
    mlflow.run('.', 'glow-jflvm', experiment_name='glow-jflvm-cv',
               parameters={'scale': 4, 'layers': layers, 'depth': depth, 'lam': lam, 'epochs': epochs,
                           'region': region, 'var': 'PRCP', 'data_lr': 'erai/daily-1deg', 'alpha': 0.0}, use_conda=False)

  0%|          | 0/2 [00:00<?, ?it/s]2020/03/09 18:34:47 INFO mlflow.projects: === Created directory /tmp/tmpybnhaogi for downloading remote URIs passed to arguments of type 'path' ===
2020/03/09 18:34:47 INFO mlflow.projects: === Running command 'python run_experiment.py glow-jflvm erai/daily-1deg --scale 4 --layers 3 --depth 4 --min-filters 32 --max-filters 256 --lam 0.1 --alpha 0.0 --prior standard --dnet-layers 3 --dnet-filters 64 --splits 3 --batch-size 10 --var PRCP --epochs 50 --validate-freq 5 --region southeast_us --mode cv --auth gcs.secret.json' in run with ID '5f6bfaa315fe4f16aaa5062ae1eb3f05' === 
2020/03/09 21:11:49 INFO mlflow.projects: === Run (ID '5f6bfaa315fe4f16aaa5062ae1eb3f05') succeeded ===
 50%|█████     | 1/2 [2:37:01<2:37:01, 9421.92s/it]2020/03/09 21:11:49 INFO mlflow.projects: === Created directory /tmp/tmpe3x7s412 for downloading remote URIs passed to arguments of type 'path' ===
2020/03/09 21:11:49 INFO mlflow.projects: === Running command 'python run_exper

KeyboardInterrupt: 

#### Fix $\lambda$, vary depth

In [3]:
layers = 3
lam = 1.0
epochs = [30]
region = 'southeast_us'
depths = [4]
# run experiment
for num_epochs, depth in tqdm(list(zip(epochs, depths))):
    mlflow.run('.', 'glow-jflvm', experiment_name='glow-jflvm-cv',
               parameters={'scale': 4, 'layers': layers, 'depth': depth, 'lam': lam, 'epochs': num_epochs,
                           'region': region, 'splits': 2, 'data_lr': 'erai/daily-1deg', 'alpha': 0.0, 'var': 'PRCP'}, use_conda=False)

  0%|          | 0/1 [00:00<?, ?it/s]2020/03/08 23:02:59 INFO mlflow.projects: === Created directory /tmp/tmprnkvrk6l for downloading remote URIs passed to arguments of type 'path' ===
2020/03/08 23:02:59 INFO mlflow.projects: === Running command 'python run_experiment.py glow-jflvm erai/daily-1deg --scale 4 --layers 3 --depth 4 --min-filters 32 --max-filters 256 --lam 1.0 --alpha 0.0 --prior standard --dnet-layers 3 --dnet-filters 64 --splits 2 --batch-size 10 --var PRCP --epochs 30 --validate-freq 5 --region southeast_us --mode cv --auth gcs.secret.json' in run with ID '3a8ee6d40f2e449fb693fb7fec8c1b13' === 
2020/03/09 00:15:29 INFO mlflow.projects: === Run (ID '3a8ee6d40f2e449fb693fb7fec8c1b13') succeeded ===
100%|██████████| 1/1 [1:12:30<00:00, 4350.44s/it]


#### Fix $\lambda$ and depth, vary $\alpha$

In [2]:
layers = 3
lam = 1.0
epochs = 20
region = 'southeast_us'
depth = 4
alphas = [0.0, 1.0E-2, 1.0]
# run experiment
for alpha in tqdm(alphas):
    mlflow.run('.', 'glow-jflvm', experiment_name='glow-jflvm-cv',
               parameters={'scale': 4, 'layers': layers, 'depth': depth, 'lam': lam, 'epochs': epochs, 'region': region, 'alpha': alpha,
                           'splits': 2, 'data_lr': 'erai/daily-1deg', 'var': 'PRCP'}, use_conda=False)

  0%|          | 0/3 [00:00<?, ?it/s]2020/03/08 15:50:20 INFO mlflow.projects: === Created directory /tmp/tmpd2cmk2_v for downloading remote URIs passed to arguments of type 'path' ===
2020/03/08 15:50:20 INFO mlflow.projects: === Running command 'python run_experiment.py glow-jflvm erai/daily-1deg --scale 4 --layers 3 --depth 4 --min-filters 32 --max-filters 256 --lam 1.0 --alpha 0.0 --prior standard --dnet-layers 3 --dnet-filters 64 --splits 2 --batch-size 10 --var PRCP --epochs 20 --validate-freq 5 --region southeast_us --mode cv --auth gcs.secret.json' in run with ID 'c3cc62d18e17436b816a5068b41214d2' === 
2020/03/08 16:13:32 ERROR mlflow.projects: === Run (ID 'c3cc62d18e17436b816a5068b41214d2') interrupted, cancelling run ===


KeyboardInterrupt: 

#### Test informative vs non-informative prior

In [ ]:
layers = 3
lam = 1.0
epochs = 20
region = 'southeast_us'
depth = 4
alpha = 0.0
priors = ['climatology', 'standard']
# run experiment
for prior in tqdm(priors):
    mlflow.run('.', 'glow-jflvm', experiment_name='glow-jflvm-4x-cv',
               parameters={'scale': 4, 'layers': layers, 'depth': depth, 'lam': lam, 'epochs': epochs,
                           'region': region, 'alpha': alpha, 'prior': prior}, use_conda=False)

In [ ]:
df_lam = pd.read_csv('data/cv_lambda_maxt.csv')

In [ ]:
df_lam.head(5)

In [ ]:
lams = [0.01, 0.1, 1.0]
avg_rmse = [df_lam.loc[df_lam['lam'] == lam]['rmse'].mean() for lam in lams]
avg_bias = [df_lam.loc[df_lam['lam'] == lam]['bias'].mean() for lam in lams]

In [ ]:
plt.figure(figsize=(2*6,4))
plt.subplot(1,2,1)
plt.plot(lams, avg_rmse)
plt.xscale('log')
plt.title('RMSE vs lambda')
plt.tight_layout()
plt.subplot(1,2,2)
plt.plot(lams, avg_bias)
plt.xscale('log')
plt.title('bias vs lambda')
plt.tight_layout()

In [ ]:
df_alpha = pd.read_csv('data/cv_alpha_maxt.csv')

In [ ]:
alphas = [0.0001, 0.001, 0.01]
avg_rmse = [df_alpha.loc[df_alpha['alpha'] == alpha]['rmse'].mean() for alpha in alphas]
avg_bias = [df_alpha.loc[df_alpha['alpha'] == alpha]['bias'].mean() for alpha in alphas]

In [ ]:
plt.figure(figsize=(2*6,4))
plt.subplot(1,2,1)
plt.plot(alphas, avg_rmse)
plt.xscale('log')
plt.title('RMSE vs alpha')
plt.tight_layout()
plt.subplot(1,2,2)
plt.plot(alphas, avg_bias)
plt.xscale('log')
plt.title('bias vs alpha')
plt.tight_layout()

In [ ]:
df_depth = pd.read_csv('data/cv_depth_maxt.csv')
df_depth.head(5)

In [ ]:
depths = [4,8,16]
avg_rmse = [df_depth.loc[df_depth['depth'] == depth]['rmse'].mean() for depth in depths]
avg_bias = [df_depth.loc[df_depth['depth'] == depth]['bias'].mean() for depth in depths]

In [ ]:
plt.figure(figsize=(2*6,4))
plt.subplot(1,2,1)
plt.plot(depths, avg_rmse)
plt.title('RMSE vs depth')
plt.tight_layout()
plt.subplot(1,2,2)
plt.plot(depths, avg_bias)
plt.title('bias vs depth')
plt.tight_layout()